In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (get_training_observations, 
                     get_training_labels)
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load

# Model specific imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import metrics
tf.get_logger().setLevel('INFO')

import optuna

In [2]:
# Set up data
x_train = get_training_observations()
y_train = get_training_labels()
le = load('../model_joblibs/final_labelEncoder.joblib')
    
y_train_transformed = le.transform(y_train['EMPO_3'])
y_train_transformed

x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train_transformed, test_size=0.2, random_state=27)   

Getting all training observations from 'metagenome_classification.db'...
Getting all training labels from 'metagenome_classification.db'...


In [3]:
K = tf.keras.backend

def build_model(
    activation='relu',
    hidden_layer_sizes=[],
    final_layer_activation='softmax',
    dropout=0.0,
    optimizer='Adam',
    learning_rate=0.01,
    kernel_regularizer=1e-5,
    bias_regularizer=1e-5,
    activity_regularizer=1e-5,
    n_classes=15
):
    """Build a multi-class logistic regression model using Keras.

    Args:
      n_classes: Number of output classes in the dataset.
      hidden_layer_sizes: A list with the number of units in each hidden layer.
      activation: The activation function to use for the hidden layers.
      optimizer: The optimizer to use (SGD, Adam).
      learning_rate: The desired learning rate for the optimizer.

    Returns:
      model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for hidden_layer_size in hidden_layer_sizes:
        if hidden_layer_size > n_classes:
            model.add(tf.keras.layers.Dense(
                hidden_layer_size,
                activation=activation,
                kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
                bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
                activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
            ))
            if dropout > 0:
                model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(
        n_classes,
        activation=final_layer_activation,
        kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
        bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
        activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
    ))
    opt = None
    if optimizer == "SGD":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise f"Unsupported optimizer, {optimizer}"
    model.compile(
        loss="categorical_crossentropy", optimizer=opt, 
        metrics=[tfa.metrics.F1Score(num_classes=15, average='weighted')]
    )
    return model
    

In [4]:
def objective_2_layer(trial, x_tr=x_tr, x_val=x_val, y_tr=y_tr, y_val=y_val):
    
    # Set up XGBoost hyperparameter space
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
        'hidden_layer_size1': trial.suggest_int('hidden_layer_size1', 0, 1024),
        'hidden_layer_size2': trial.suggest_int('hidden_layer_size2', 0, 1024),
        'kernel_regularizer': trial.suggest_float('kernel_regularizer', 1e-10, 1e-4),
        'bias_regularizer': trial.suggest_float('bias_regularizer', 1e-10, 1e-4),
        'activity_regularizer': trial.suggest_float('activity_regularizer', 1e-10, 1e-4),
        'dropout': trial.suggest_float('dropout', 0, 0.1)
    }
    
    hidden_layer_sizes = [params.pop('hidden_layer_size1'), params.pop('hidden_layer_size2')]
    
    y_tr_one_hot = K.one_hot(K.cast(y_tr, dtype='int64'), 15)
    y_val_one_hot = K.one_hot(K.cast(y_val, dtype='int64'), 15)
    
    # Create model
    nn = build_model(
        **params,
        n_classes=15,
        optimizer='Adam',
        final_layer_activation='softmax',
        hidden_layer_sizes=hidden_layer_sizes
    )
    
    # Fit model
    nn.fit(x=x_tr,
           y=y_tr_one_hot,
           epochs=50,
           batch_size=64,
           verbose=0)
    
    evaluation = nn.evaluate(x=x_val, y=y_val_one_hot, verbose=0, return_dict=True)
    f1_score = evaluation['f1_score']
    loss = evaluation['loss']
    
    print(f'F1 Score: {f1_score}; Loss={loss}')
    
    return f1_score
    

In [ ]:
study_nn_2 = optuna.create_study(direction='maximize')
study_nn_2.optimize(objective_2_layer, n_trials=200)

print('Number of finished trials:', len(study_nn_2.trials))
print('Best trial:', study_nn_2.best_trial.params)

[I 2022-08-03 01:34:32,009] A new study created in memory with name: no-name-a9133337-b67e-42bf-8233-7059ed00cc8a


Metal device set to: Apple M1 Pro


2022-08-03 01:34:32.011185: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 01:34:32.011302: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-03 01:34:32.272870: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 01:34:33.093874: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 01:34:52.553208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 01:34:52,799] Trial 0 finished with value:

F1 Score: 0.8127579092979431; Loss=0.8609961271286011


2022-08-03 01:34:53.618974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 01:35:14.825515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 01:35:15,059] Trial 1 finished with value: 0.9067314863204956 and parameters: {'learning_rate': 0.001848870210055203, 'hidden_layer_size1': 604, 'hidden_layer_size2': 1019, 'kernel_regularizer': 5.788444265722835e-06, 'bias_regularizer': 4.794195273557686e-05, 'activity_regularizer': 4.2617283091840946e-05, 'dropout': 0.053811190216511234}. Best is trial 1 with value: 0.9067314863204956.


...

2022-08-03 02:13:07.484594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 02:13:47.749328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 02:13:49,650] Trial 59 finished with value: 0.8787274360656738 and parameters: {'learning_rate': 0.006207876921667037, 'hidden_layer_size1': 286, 'hidden_layer_size2': 210, 'kernel_regularizer': 1.8876524391765376e-05, 'bias_regularizer': 5.97887702802058e-05, 'activity_regularizer': 2.401375791682321e-06, 'dropout': 0.07586856895767112}. Best is trial 31 with value: 0.9366583824157715.


F1 Score: 0.8787274360656738; Loss=0.5411748290061951


2022-08-03 02:13:51.778318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 02:14:57.426566: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 02:14:59,997] Trial 60 finished with value: 0.8376790285110474 and parameters: {'learning_rate': 0.007969313906468652, 'hidden_layer_size1': 153, 'hidden_layer_size2': 496, 'kernel_regularizer': 7.893671023847936e-05, 'bias_regularizer': 3.562875041936652e-05, 'activity_regularizer': 5.560123981019426e-05, 'dropout': 0.04261163035146272}. Best is trial 31 with value: 0.9366583824157715.


F1 Score: 0.8376790285110474; Loss=0.7702682614326477


2022-08-03 02:15:02.875255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


* Trial 31 finished with value: 0.9366583824157715 and parameters: {'learning_rate': 0.0057554337551177105, 'hidden_layer_size1': 227, 'hidden_layer_size2': 786, 'kernel_regularizer': 1.0103547773871587e-07, 'bias_regularizer': 3.174256984709423e-05, 'activity_regularizer': 9.21848377690784e-06, 'dropout': 0.043861145616068783}. Best is trial 31 with value: 0.9366583824157715.